### 1. Imports et configuration

In [110]:
import pandas as pd
import numpy as np
import os

# Configuration pour une meilleure lisibilité
pd.set_option('display.max_columns', None)

### 2. Création des dossiers


In [111]:
# Créer des dossiers pour les données traitées
os.makedirs('data_processed', exist_ok=True)

### 3. Chargement des données


In [112]:

print("Chargement des données...")
allocine_df = pd.read_parquet('data/allocine_spider.parquet')
frequentation_df = pd.read_parquet('data/frequentation_data.parquet')
parts_marche_df = pd.read_parquet('data/parts_marche_data.parquet')
actor_df = pd.read_parquet('data/actor_JpboxSpider.parquet')

Chargement des données...


In [113]:
# Afficher les dimensions
print(f"Allociné : {allocine_df.shape}")
print(f"Fréquentation : {frequentation_df.shape}")
print(f"Parts de marché : {parts_marche_df.shape}")
print(f"Acteurs : {actor_df.shape}")

Allociné : (9158, 18)
Fréquentation : (1526, 5)
Parts de marché : (202, 8)
Acteurs : (499, 8)


In [114]:
# Création d'une variable indicatrice pour la période COVID
allocine_df['year'] = pd.to_datetime(allocine_df['date'], errors='coerce').dt.year
allocine_df['covid_period'] = allocine_df['year'].apply(lambda x: 1 if 2020 <= x <= 2021 else 0)

# Vérification de la nouvelle colonne
print("Aperçu des données avec la variable COVID :")
print(allocine_df[['year', 'covid_period']].head())



Aperçu des données avec la variable COVID :
     year  covid_period
0  2023.0             0
1  2020.0             1
2  2024.0             0
3  2022.0             0
4  2020.0             1


### 4. Nettoyage du dataset Allociné


In [115]:
#### 4.1 Suppression des doublons

allocine_df = allocine_df.drop_duplicates(subset=['title', 'date'], keep='last')
print(f"Allociné après suppression des doublons : {allocine_df.shape}")

Allociné après suppression des doublons : (9158, 20)


In [116]:
#### 4.2 Gestion des valeurs manquantes

allocine_df['critics_score'] = pd.to_numeric(allocine_df['critics_score'], errors='coerce')
allocine_df['viewers_score'] = pd.to_numeric(allocine_df['viewers_score'], errors='coerce')

allocine_df['critics_score'].fillna(allocine_df['critics_score'].median())
allocine_df['viewers_score'].fillna(allocine_df['viewers_score'].median())

allocine_df['length'] = pd.to_numeric(allocine_df['length'], errors='coerce')
allocine_df['length'].fillna(allocine_df['length'].median())

0       116.0
1        90.0
2        84.0
3       120.0
4       137.0
        ...  
9153    127.0
9154    114.0
9155     96.0
9156     77.0
9157     98.0
Name: length, Length: 9158, dtype: float64

In [117]:
# for col in ['genre', 'actors', 'directors', 'nationality', 'langage']:
#     allocine_df[col] = (
#         allocine_df[col]
#         .str.replace(r'[;,/]+', '|', regex=True)  # Remplace virgules, points-virgules, etc. par |
#         .str.replace(r'\s*\|\s*', '|', regex=True)  # Nettoie les espaces autour des |
#     )
allocine_df['genre'] = allocine_df['genre'].str.split('|')
allocine_df['actors'] = allocine_df['actors'].str.split('|')
allocine_df['nationality'] = allocine_df['nationality'].str.split('|')
allocine_df['langage'] = allocine_df['langage'].str.split('|')
allocine_df['directors'] = allocine_df['directors'].str.split('|')

for col in ['genre', 'langage', 'nationality', 'actors']:
    allocine_df[col] = allocine_df[col].mask(allocine_df[col].isna(), ['no value'])

In [118]:
allocine_df.dtypes

actors                          object
critics_score                  float64
date                            object
directors                       object
editor                          object
french_boxoffice                 int64
french_first_week_boxoffice    float64
french_visa                     object
genre                           object
langage                         object
length                         float64
nationality                     object
title                           object
url                             object
us_boxoffice                   float64
us_first_week_boxoffice        float64
viewers_score                  float64
vo_title                        object
year                           float64
covid_period                     int64
dtype: object

In [119]:

# Étape 1 : Compter les films avant tout traitement
print(f"Films totaux au départ : {len(allocine_df)}")

# Étape 2 : Conversion de la date en datetime et suppression des dates invalides
allocine_df['date'] = pd.to_datetime(allocine_df['date'], errors='coerce', format='%Y-%m-%d')
allocine_df = allocine_df[allocine_df['date'].notna()]  # Supprimer les dates invalides

invalid_dates_count = len(allocine_df) - len(allocine_df[allocine_df['date'].notna()])
print(f"Dates invalides (supprimées) : {invalid_dates_count}")

# Étape 3 : Filtrage par année (directement via la colonne datetime)
min_year = 2001
mask = allocine_df['date'].dt.year >= min_year  # Extraction de l'année directement dans le filtre

print(f"\nAvant filtrage par année >= {min_year} : {len(allocine_df)} films")
allocine_df = allocine_df[mask]
print(f"Après filtrage : {len(allocine_df)} films restants")


Films totaux au départ : 9158
Dates invalides (supprimées) : 0

Avant filtrage par année >= 2001 : 9154 films
Après filtrage : 9115 films restants


In [120]:
#### 4.5 Suppression des outliers

allocine_df = allocine_df[allocine_df['french_boxoffice'].notna()]

q_low = allocine_df['french_boxoffice'].quantile(0.20)
allocine_df = allocine_df[
    (allocine_df['french_boxoffice'] > q_low)
]
print(f'La valeur correspondant au 5ème percentile est : {q_low}')

La valeur correspondant au 5ème percentile est : 6487.200000000003


In [121]:
#### 4.6 Vérification finale

print("Aperçu Allociné nettoyé :")
print(allocine_df.info())
print(allocine_df.head())

Aperçu Allociné nettoyé :
<class 'pandas.core.frame.DataFrame'>
Index: 7292 entries, 0 to 9157
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   actors                       7292 non-null   object        
 1   critics_score                7291 non-null   float64       
 2   date                         7292 non-null   datetime64[ns]
 3   directors                    7292 non-null   object        
 4   editor                       7292 non-null   object        
 5   french_boxoffice             7292 non-null   int64         
 6   french_first_week_boxoffice  7291 non-null   float64       
 7   french_visa                  7292 non-null   object        
 8   genre                        7292 non-null   object        
 9   langage                      7292 non-null   object        
 10  length                       7287 non-null   float64       
 11  nationality           

### 5. Nettoyage du dataset Fréquentation


In [122]:
#### 5.1 Conversion des colonnes numériques

for col in ['Année Courante', 'Année Précédente', 'Évolution (%)']:
    frequentation_df[col] = pd.to_numeric(frequentation_df[col], errors='coerce')

frequentation_df = frequentation_df[frequentation_df['Année Courante'] >= 0]

In [123]:
#### 5.2 Imputation des valeurs manquantes

frequentation_df['Évolution (%)'].fillna(0)

0       -7.8
1      -16.4
2       -4.8
3      -35.5
4      -17.1
        ... 
1521     8.8
1522    -3.4
1523   -12.1
1524    16.3
1525     6.8
Name: Évolution (%), Length: 1486, dtype: float64

In [124]:
#### 5.3 Suppression des doublons

frequentation_df = frequentation_df.drop_duplicates(
    subset=['Date Article', 'Période'], keep='last'
)

In [125]:
#### 5.4 Vérification finale

print("Aperçu Fréquentation nettoyé :")
print(frequentation_df.info())
print(frequentation_df.head())

Aperçu Fréquentation nettoyé :
<class 'pandas.core.frame.DataFrame'>
Index: 1356 entries, 0 to 1525
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date Article      1356 non-null   object 
 1   Période           1356 non-null   object 
 2   Année Courante    1356 non-null   float64
 3   Année Précédente  1302 non-null   float64
 4   Évolution (%)     1270 non-null   float64
dtypes: float64(3), object(2)
memory usage: 63.6+ KB
None
  Date Article               Période  Année Courante  Année Précédente  \
0   2024-05-03               Janvier           13.67             14.82   
1   2024-05-03               Février           15.07             18.03   
2   2024-05-03                  Mars           15.00             15.76   
3   2024-05-03                 Avril           12.00             18.61   
4   2024-05-03  quatre premiers mois           55.75             67.21   

   Évolution (%)  
0           -7.8 

### 6. Nettoyage du dataset Parts de marché

In [126]:
#### 6.1 Conversion des colonnes numériques

for col in parts_marche_df.columns:
    if col not in ['Date Article', 'Période']:
        parts_marche_df[col] = pd.to_numeric(parts_marche_df[col], errors='coerce')

In [127]:
#### 6.2 Suppression des valeurs aberrantes

parts_marche_df = parts_marche_df[
    (parts_marche_df['Films Français Année Courante'] <= 100) &
    (parts_marche_df['Films Américains Année Courante'] <= 100) &
    (parts_marche_df['Autres Films Année Courante'] <= 100)
]

In [128]:
#### 6.3 Imputation des valeurs manquantes

parts_marche_df.ffill()

,Date Article,Période,Films Français Année Courante,Films Français Année Précédente,Films Américains Année Courante,Films Américains Année Précédente,Autres Films Année Courante,Autres Films Année Précédente
1,2024-05-03,quatre premiers mois,44.5,47.8,29.9,39.5,25.7,12.7
2,2024-05-03,année glissante (de mai n-1 à avril n),38.5,41.2,38.2,44.0,23.3,14.8
4,2024-04-02,trois premiers mois,46.6,50.2,25.4,37.1,28.0,12.7
5,2024-04-02,année glissante (de avril n-1 à mars n),38.9,41.4,38.5,42.5,22.6,16.1
7,2023-11-02,dix premiers mois,38.5,42.0,45.9,36.9,15.6,21.0
...,...,...,...,...,...,...,...,...
193,2010-12-29,Dix premiers mois,36.8,47.0,48.5,44.3,14.8,8.7
195,2010-12-08,11 premiers mois,36.4,36.6,47.6,48.9,16.0,14.5
197,2010-12-29,Sept premiers mois,35.2,50.1,48.0,42.3,16.8,7.6
199,2010-12-29,10 premiers mois,34.7,36.5,49.5,47.9,15.7,15.7


In [129]:
#### 6.4 Suppression des doublons

parts_marche_df = parts_marche_df.drop_duplicates(
    subset=['Date Article', 'Période'], keep='last'
)

In [130]:
#### 6.5 Vérification finale

print("Aperçu Parts de marché nettoyé :")
print(parts_marche_df.info())
print(parts_marche_df.head())

Aperçu Parts de marché nettoyé :
<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, 1 to 201
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Date Article                       126 non-null    object 
 1   Période                            126 non-null    object 
 2   Films Français Année Courante      126 non-null    float64
 3   Films Français Année Précédente    107 non-null    float64
 4   Films Américains Année Courante    126 non-null    float64
 5   Films Américains Année Précédente  107 non-null    float64
 6   Autres Films Année Courante        126 non-null    float64
 7   Autres Films Année Précédente      107 non-null    float64
dtypes: float64(6), object(2)
memory usage: 8.9+ KB
None
  Date Article                                  Période  \
1   2024-05-03                     quatre premiers mois   
2   2024-05-03   année glissante (de mai n-1 à avril 

### 7. Nettoyage du dataset Acteurs

In [131]:
#### 7.1 Suppression des doublons

actor_df = actor_df.drop_duplicates(subset=['name', 'nationality'], keep='last')
print(f"Acteurs après suppression des doublons : {actor_df.shape}")

Acteurs après suppression des doublons : (499, 8)


In [132]:
#### 7.2 Gestion des valeurs manquantes

actor_df['boxoffice_average'] = pd.to_numeric(actor_df['boxoffice_average'], errors='coerce')
actor_df['boxoffice_total'] = pd.to_numeric(actor_df['boxoffice_total'], errors='coerce')

actor_df['boxoffice_average'].fillna(0)
actor_df['boxoffice_total'].fillna(0)

0       3360134
1       3083328
2       3033394
3       2643888
4       1579207
         ...   
494    76669820
495    76134103
496    73392185
497    73375624
498    73021666
Name: boxoffice_total, Length: 499, dtype: int64

In [133]:
#### 7.3 Vérification des types de données

actor_df['amount_of_film_played_in'] = pd.to_numeric(actor_df['amount_of_film_played_in'], errors='coerce')

In [134]:
#### 7.4 Vérification finale

print("Aperçu Acteurs nettoyé :")
print(actor_df.info())
print(actor_df.head())

Aperçu Acteurs nettoyé :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   amount_of_film_played_in      499 non-null    int64 
 1   boxoffice_average             499 non-null    int64 
 2   boxoffice_average_first_role  499 non-null    int64 
 3   boxoffice_total               499 non-null    int64 
 4   boxoffice_total_first_role    499 non-null    int64 
 5   name                          499 non-null    object
 6   nationality                   499 non-null    object
 7   rank                          499 non-null    int64 
dtypes: int64(6), object(2)
memory usage: 31.3+ KB
None
   amount_of_film_played_in  boxoffice_average  boxoffice_average_first_role  \
0                        15             197655                        284469   
1                        18              99462                         9479

### 8. Sauvegarde des données nettoyées


In [135]:

# allocine_df.to_parquet('data_processed/allocine_cleaned.parquet', index=False)
allocine_df.to_csv('data_processed/allocine_cleaned.csv', index=False)

frequentation_df.to_parquet('data_processed/frequentation_cleaned.parquet', index=False)
frequentation_df.to_csv('data_processed/frequentation_cleaned.csv', index=False)

parts_marche_df.to_parquet('data_processed/parts_marche_cleaned.parquet', index=False)
parts_marche_df.to_csv('data_processed/parts_marche_cleaned.csv', index=False)

actor_df.to_parquet('data_processed/actor_cleaned.parquet', index=False)
actor_df.to_csv('data_processed/actor_cleaned.csv', index=False)

print("Données nettoyées sauvegardées avec succès !")

Données nettoyées sauvegardées avec succès !
